In [ ]:
import yfinance as yf
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
nse = "TATAMOTORS.NS"

In [ ]:
start_date=datetime.datetime(2024,4,1)
end_date=datetime.datetime(2025,4,30)

In [ ]:
stock = yf.download(nse, start_date,end_date)
stock.index = pd.to_datetime(stock.index)


[*********************100%***********************]  1 of 1 completed


In [ ]:
print(stock.head())


Price              Close          High           Low          Open  \
Ticker     TATAMOTORS.NS TATAMOTORS.NS TATAMOTORS.NS TATAMOTORS.NS   
Date                                                                 
2024-04-01    989.197388   1006.743218    984.412167    996.923545   
2024-04-02   1001.559265   1004.898935    990.343875    995.926623   
2024-04-03   1006.045471   1011.329154    989.496503   1000.911290   
2024-04-04   1008.487854   1013.871266    995.079281   1012.874342   
2024-04-05   1004.001709   1009.385121   1001.310064   1008.487865   

Price             Volume  
Ticker     TATAMOTORS.NS  
Date                      
2024-04-01       8629407  
2024-04-02       7995931  
2024-04-03       8040366  
2024-04-04       9138276  
2024-04-05       4519120  


In [ ]:
df = yf.download(
    tickers=nse,
    start=start_date,
    end=end_date,
    interval="1d",
    group_by="ticker",

)
df

[*********************100%***********************]  1 of 1 completed


Ticker     TATAMOTORS.NS                                                 
Price               Open         High          Low        Close    Volume
Date                                                                     
2024-04-01    996.923545  1006.743218   984.412167   989.197388   8629407
2024-04-02    995.926623  1004.898935   990.343875  1001.559265   7995931
2024-04-03   1000.911290  1011.329154   989.496503  1006.045471   8040366
2024-04-04   1012.874342  1013.871266   995.079281  1008.487854   9138276
2024-04-05   1008.487865  1009.385121  1001.310064  1004.001709   4519120
...                  ...          ...          ...          ...       ...
2025-04-23    635.950012   661.500000   633.349976   659.900024  23390570
2025-04-24    660.099976   671.000000   660.099976   668.349976  15172229
2025-04-25    668.349976   673.000000   651.500000   655.250000  13207043
2025-04-28    654.299988   669.599976   652.549988   668.150024  10243493
2025-04-29    671.450012   682.900024   661.599976   665.599976  13754685

[266 rows x 5 columns]

In [ ]:
df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 266 entries, 2024-04-01 to 2025-04-29
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   (TATAMOTORS.NS, Open)    266 non-null    float64
 1   (TATAMOTORS.NS, High)    266 non-null    float64
 2   (TATAMOTORS.NS, Low)     266 non-null    float64
 3   (TATAMOTORS.NS, Close)   266 non-null    float64
 4   (TATAMOTORS.NS, Volume)  266 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 12.5 KB


In [ ]:
fig = go.Figure(data=[
    go.Candlestick(x=df.index, open=df[nse]['Open'], high=df[nse]['High'], low=df[nse]['Low'], close=df[nse]['Close']),
    go.Scatter(x=['2024-06-05'], y=[3881], mode='markers+text', marker=dict(size=12, color='red', symbol='circle'), text=["Significant Drop"], textposition="top center", name="Noteworthy Event"),
    go.Scatter(x=['2025-03-02', '2025-03-02'], y=[df[nse]['Low'].min() - 10, df[nse]['High'].max() + 10], mode='lines', line=dict(color='green', width=4, dash='dot'), name='Potential Reversal Point')
])

fig.update_layout(title={'text': f"Financial Performance: {nse}", 'x': 0.5, 'xanchor': 'center'},
                  xaxis_title='Trading Date', yaxis_title='Value', hovermode="x unified", template="plotly_dark", xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
df[(nse, 'Daily Return')] = df[nse]['Close'].pct_change()
df[(nse, 'Log Return')] = np.log(df[nse]['Close'] / df[nse]['Close'].shift(1))

fig = make_subplots(rows=1, cols=2, subplot_titles=("Simple Return", "Log Return"))

fig.add_trace(go.Scatter(x=df.index, y=df[(nse, 'Daily Return')], mode='lines', name='Daily Return'), row=1, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df[(nse, 'Log Return')], mode='lines', name='Log Return'), row=1, col=2)

fig.update_layout(title_text="Problem 2", title_x=0.5, height=500, showlegend=False, template="plotly_dark")
fig.show()


Volatility Estimation

In [ ]:
df.loc[:, (nse, 'Rolling Std Dev')] = df[nse]['Daily Return'].rolling(window=14).std()
fig1 = px.line(df, x=df.index, y=df[nse]['Rolling Std Dev'])
fig1.update_layout(title='Rolling 14-Day Standard Deviation', xaxis_title='Date', yaxis_title='Standard Deviation')
fig1.show()
fig2 = go.Figure(data=[
    go.Candlestick(x=df.index,
                   open=df[nse]['Open'],
                   high=df[nse]['High'],
                   low=df[nse]['Low'],
                   close=df[nse]['Close'])
              ])
fig2.update_layout(title=f"Candlestick Chart - {nse}", xaxis_title='Date', yaxis_title='Price')
fig2.show()
#as it was written to plot it along with price, I have plotted price by candlestick chart below it

Simple vs Log Returns

**Simple returns** are calculated as the percentage change in price:  
R_t = (Pt-Pt-1)/Pt-1

**Log returns** are calculated using natural logarithms:  
Rt=ln(Pt/Pt-1)

For daily data with small price changes, both returns look very similar. This is because when changes are small, ln(1 + x) approx x. However, during periods of large price movements (e.g., earnings surprises or market crashes), log returns tend to differ more noticeably due to their compounding nature.

Log returns are **time additive**, meaning that the return over multiple periods can be obtained by summing the individual log returns — a useful property in financial modeling and portfolio optimization.

In contrast, simple returns are easier to interpret intuitively as percentage gains or losses. Both are valid tools, and the choice depends on the context — log returns are often used in quantitative models, while simple returns are more common in performance reporting.


**Problem Statement - 3**

In [ ]:
from scipy.stats import binom

In [ ]:
# Determine the direction of daily price movement
df.loc[:, (nse, 'Market_Direction')] = df[nse]['Daily Return'].apply(lambda x: 'UP' if x > 0 else 'DOWN')

# Calculate the total number of valid trading days
total_trading_periods = df[nse]['Market_Direction'].count()

# Count the number of 'UP' days
favorable_days = (df[nse]['Market_Direction'] == 'UP').sum()

# Calculate the probability of an 'UP' day
prob_up_movement = favorable_days / total_trading_periods

print(f'P(UP) = {prob_up_movement:.3f}')

P(UP) = 0.462


In [ ]:
# Calculate theoretical probabilities based on P(UP) = 0.6
theoretical_exact_6_up = binom.pmf(k=6, n=10, p=0.6)
theoretical_at_least_8_up = 1 - binom.cdf(k=7, n=10, p=0.6)

# Calculate actual probabilities using the derived P_UP from stock data
actual_exact_6_up = binom.pmf(k=6, n=10, p=P_UP)
actual_at_least_8_up = 1 - binom.cdf(k=7, n=10, p=P_UP)

print(f"Theoretical Probability of exactly 6 up days in 10: {theoretical_exact_6_up:.3f}")
print(f"Theoretical Probability of at least 8 up days in 10: {theoretical_at_least_8_up:.3f}")
print(f"Actual Probability of exactly 6 up days in 10: {actual_exact_6_up:.3f}")
print(f"Actual Probability of at least 8 up days in 10: {actual_at_least_8_up:.3f}")

Theoretical Probability of exactly 6 up days in 10: 0.251
Theoretical Probability of at least 8 up days in 10: 0.167
Actual Probability of exactly 6 up days in 10: 0.171
Actual Probability of at least 8 up days in 10: 0.033


In [ ]:
coin_flips=np.random.binomial(n=10, p=0.6, size=1000)

In [ ]:
print(coin_flips)

[ 7  3  6  8  7  6  3  9  3  7  8 10  6  6  7  4  5  5  9  7  6  5  7  5
  4  7  5  8  4  5  7  7  6  5  5  2  8  7  7  6  4  7  8  4  7  5  7  8
  7  6  7  6  6  4  6  6  4  9  5  6  4  4  7  6  8  4  9  4  7  8  6  8
  7  3  7  8  8  6  6  6  5  4  8  9  5  5  7  5  4  8  7  5  6  7  4  7
  6  7  5  7  7  7  7  4  8  1  7  5  6  5  6  8  5  8  7  6  5  8  8  7
  4  5  6  6  6  9  6  5  5  4  5  4  8  6  6  8  4  4  4  6  6  9  7  6
  8  4  8  5  5  7  3  8  3  6  6  3  4  8  5  6  8  9  3  7  6  9  6  4
  7  4  4  9  6  8  7  8  6  4  6  7  4  5  7  6  6  8  6  4  6  5  5  7
  7  6  7  7 10  5  7  7 10  5  5  7  5  5  7  7  7  4  6  8  6  3  6  5
  5  8  6  6  7  5  7  3  6  8  8  5  2  7  6  8  6  6  8  7  7  8  5  4
  4  4  6  5  4  7  5  7  4  6  8  7  7  6  6  7  6  6  7  9  8  5  5  8
  4  4  6  4  7  6  6  6  8  4  5  9  6  5  8  5  4  5  3  6  5  1  5  7
  7  5  6  6  5  6  6  6  4  5  5  7  8  7  7  5  3  5  7  6  4  4  8  6
  6  7  5  8  5  4  1  7  7  7  8  4  7  7  6  7  3

In [ ]:
# Calculate the simulated probability of observing exactly 6 successful outcomes
simulated_frequency_of_6 = np.mean(coin_flips == 6)

# Calculate the simulated probability of observing 8 or more successful outcomes
simulated_frequency_of_8_or_more = np.mean(coin_flips >= 8)

print(simulated_frequency_of_6)
print(simulated_frequency_of_8_or_more)

0.255
0.161


**Part 2: The Paradox**

In [ ]:
# Calculate the mean daily return for 'UP' market days
average_upward_return = df[nse].loc[df[nse]['Market_Direction'] == 'UP', 'Daily Return'].mean()

# Calculate the mean daily return for 'DOWN' market days
average_downward_return = df[nse].loc[df[nse]['Market_Direction'] == 'DOWN', 'Daily Return'].mean()

print(f'Mean return on upward days: {average_upward_return}')
print(f'Mean return on downward days: {average_downward_return}')

Mean return on upward days: 0.014475132220691165
Mean return on downward days: -0.014946548761359634


In [ ]:
results = []
data = df[nse][['Daily Return', 'Market_Direction']].copy()
for i in range(len(data) - 9):
    window = data.iloc[i:i+10]#rolling 10 day window

    up_returns = window[window['Market_Direction'] == 'UP']['Daily Return']
    down_returns = window[window['Market_Direction'] == 'DOWN']['Daily Return']

    if len(up_returns) >= 6:
        up_avg = up_returns.mean()
        down_avg = down_returns.mean()

        if down_avg < up_avg:
            results.append((data.index[i], data.index[i+9]))
# Displaying the date windows in which even with P(UP)>=0.6 we can lose money
for start, end in results:
    print(f"Window: {start.date()} to {end.date()}")

Window: 2024-04-01 to 2024-04-15
Window: 2024-04-02 to 2024-04-16
Window: 2024-04-10 to 2024-04-25
Window: 2024-04-16 to 2024-04-30
Window: 2024-04-18 to 2024-05-02
Window: 2024-04-19 to 2024-05-03
Window: 2024-04-22 to 2024-05-06
Window: 2024-04-23 to 2024-05-07
Window: 2024-04-24 to 2024-05-08
Window: 2024-04-25 to 2024-05-09
Window: 2024-04-26 to 2024-05-10
Window: 2024-04-29 to 2024-05-13
Window: 2024-04-30 to 2024-05-14
Window: 2024-05-02 to 2024-05-15
Window: 2024-05-06 to 2024-05-17
Window: 2024-05-07 to 2024-05-21
Window: 2024-05-08 to 2024-05-22
Window: 2024-05-09 to 2024-05-23
Window: 2024-05-29 to 2024-06-11
Window: 2024-05-30 to 2024-06-12
Window: 2024-05-31 to 2024-06-13
Window: 2024-06-03 to 2024-06-14
Window: 2024-06-04 to 2024-06-18
Window: 2024-06-05 to 2024-06-19
Window: 2024-06-06 to 2024-06-20
Window: 2024-06-07 to 2024-06-21
Window: 2024-06-26 to 2024-07-09
Window: 2024-06-27 to 2024-07-10
Window: 2024-06-28 to 2024-07-11
Window: 2024-07-04 to 2024-07-18
Window: 20


**Part 3**

Let's break down the profitability of this bet. Your **net return** is determined by the potential winnings from "up" days balanced against losses from "down" days.

The formula for net return is:
$$( \text{winnings per UP day} \times P(\text{UP}) ) - ( \text{losses per DOWN day} \times P(\text{DOWN}) ) = \text{Net Return}$$

Given the bet terms:

You win 100 dollar when your stock goes up.
You lose 150 dollar when your stock goes down
We also know that $P(\text{DOWN}) = 1 - P(\text{UP})$.

Substituting these values into the formula:
$$\text{Net Return} = (100 \times P(\text{UP})) - (150 \times (1 - P(\text{UP})))$$
$$\text{Net Return} = 100 \times P(\text{UP}) - 150 + 150 \times P(\text{UP})$$
$$\text{Net Return} = 250 \times P(\text{UP}) - 150$$

For this bet to be profitable, your **net return must be positive**:
$$250 \times P(\text{UP}) - 150 > 0$$
$$250 \times P(\text{UP}) > 150$$
$$P(\text{UP}) > \frac{150}{250}$$
$$P(\text{UP}) > 0.6$$

